In [1]:
%pylab qt5

Populating the interactive namespace from numpy and matplotlib


In [2]:
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5

In [3]:
from importlib import reload
import lib2

In [4]:
from lib import data_management as dm

In [5]:
if_freq = 50e6

calibrations = (dm.load_IQMX_calibration_database("TEST", -20))
ro_cal = calibrations.get(frozenset(dict(lo_power=0, ssb_power=-30, lo_frequency=7.047e9, 
                                      if_frequency=0, waveform_resolution=1).items()))
# ro_pb = PulseBuilder(ro_cal)

calibrations = (dm.load_IQMX_calibration_database("TESTQ", -6))
q_cal = calibrations.get(frozenset(dict(lo_power=15, ssb_power=-10, lo_frequency=7.9e9+if_freq, 
                                      if_frequency=if_freq, waveform_resolution=0.1).items()))
# q_pb = PulseBuilder(q_cal)

# Spectroscopy

## Single-tone spectroscopy

In [38]:
reload(lib2.Measurement)

<module 'lib2.Measurement' from 'C:\\Users\\PAINMASTER\\Measurement\\lib2\\Measurement.py'>

In [10]:
# reload(lib2.SingleToneSpectroscopy)
from lib2.SingleToneSpectroscopy import *

#### No mixer 

In [ ]:
STS = SingleToneSpectroscopy("IV-anticrossing", "Xmon Al BMSTU S444 2", vna, "current", current_src.set_current, 0)
vna_parameters = {"bandwidth":250, "freq_limits":(7030e6, 7070e6), "nop":101, "power":-40, "averages":1}
STS.setup_control_parameters(vna_parameters, linspace(-1e-3, 1e-3, 100))
result = STS.launch()
result.visualize()

In [ ]:
result.visualize()

In [ ]:
result.save()

#### Through mixer

In [30]:
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(100).build())

In [31]:
STS = SingleToneSpectroscopy("IV-anticrossing-through-mixer", "Xmon Al BMSTU S444 2", "current", current_src.set_current)
vna_parameters = {"bandwidth":500, "freq_limits":(7030e6, 7070e6), "nop":101, "power":-10, "averages":1}
currents = linspace(0.1e-3, 2e-3, 100)
STS.setup_control_parameters(vna_parameters, currents)
result = STS.launch()
current_src.set_current(0)
result.visualize()

The device vna2 is detected as PNA-L2
Started at:  2017-04-25 21:05:38.633681
Time left: 0 h 0 m 40.99 s, current: 1.000e-04, average cycle time: 0.41 s          

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s, current: 2.000e-03, average cycle time: 0.29 s            

In [31]:
result.visualize()

In [33]:
result.save()

## Two-tone spectroscopy

In [10]:
# reload(lib2.Measurement)
# reload(lib2.TwoToneSpectroscopyBase)
# reload(lib2.TwoToneSpectroscopy)
from lib2.TwoToneSpectroscopy import *

In [16]:
q_awg.output_pulse_sequence(q_pb.add_dc_pulse(100,5).build())

#### Qubit spectrum

In [13]:
TTS = FluxTwoToneSpectroscopy("VI-2tone", "Xmon Al BMSTU S444 2")
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":-10, "averages":1}
mw_src_parameters = {"power":-5, "frequency":1j}
mw_src_frequencies = linspace(7e9, 8.5e9, 100)
currents = linspace(0.507e-3, 1.57e-3, 100)
TTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, currents)

Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at qubit's sweet spot (1.04 mA)
Detected frequency is 7.04713 GHz, at 0.23 mU and 42.82 degrees


In [14]:
result = TTS.launch()

Started at:  2017-04-25 22:07:43.336722
Time left: 0 h 27 m 16.94 s, parameter value: 5.070e-04, average cycle time: 0.16 s          

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 20 m 1.3 s, parameter value: 6.895e-04, average cycle time: 0.15 s            

In [15]:
result.visualize()

In [91]:
result.save()

#### Linewidth from excitation power

In [16]:
PTS = PowerTwoToneSpectroscopy("VI-powerscan", "Xmon Al BMSTU S444 2")
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":-20, "averages":1}
mw_src_parameters = {"power":-1, "frequency":-1}
mw_src_frequencies = linspace(7.88e9, 7.92e9,200)
mw_src_powers = linspace(-20, 15, 41)
current = 1.04e-3
PTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, mw_src_powers, current)
PTS._measurement_result.set_phase_units("deg")

Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of 1.04 mA
Detected frequency is 7.04700 GHz, at 0.20 mU and 44.41 degrees


In [17]:
result = PTS.launch()

Started at:  2017-04-25 22:12:39.898684
Time left: 0 h 23 m 17.5 s, parameter value: -2.000e+01, average cycle time: 0.17 s           

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 18 m 41.69 s, parameter value: -1.825e+01, average cycle time: 0.15 s          

In [ ]:
vna.set_parameters(PTS._pre_measurement_vna_parameters)

In [106]:
result.visualize()

In [103]:
result.save()

In [ ]:
mxg.set_frequency(7.898e9)
mxg.set_power(-30)
mxg.set_output_state("OFF")
vna.prepare_for_stb(); vna.sweep_single(); vna.wait_for_stb()
ground_state_sdata = vna.get_sdata()

mxg.set_output_state("ON")
vna.prepare_for_stb(); vna.sweep_single(); vna.wait_for_stb()
mixed_state_sdata = vna.get_sdata()

In [ ]:
plot(abs(ground_state_sdata))
plot(abs(mixed_state_sdata))
plot((abs(ground_state_sdata)[10:]+abs(ground_state_sdata)[:-10])/2)

#### AC Stark

In [20]:
ASTS = AcStarkTwoToneSpectroscopy("VI-ac-stark", "Xmon Al BMSTU S444 2")
vna_parameters = {"bandwidth":1000, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":-10, "averages":100}
mw_src_parameters = {"power":-15, "frequency":-1}
mw_src_frequencies = linspace(7.85e9, 7.9e9, 100)
vna_powers = linspace(-30, 0, 31)
current = 1.04e-3
ASTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, vna_powers, current)

Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of 1.04 mA
Detected frequency is 7.04713 GHz, at 0.23 mU and 45.58 degrees


In [21]:
result = ASTS.launch()

Started at:  2017-04-25 22:15:13.034443
Time left: 0 h 27 m 19.46 s, parameter value: -3.000e+01, average cycle time: 0.53 s          

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s, parameter value: 0.000e+00, average cycle time: 0.16 s              

In [22]:
result.visualize()

In [23]:
result.save()

__________________________

# Time-domain measurements

#### Check mixers:

In [175]:
q_lo = mxg
q_lo.set_output_state("ON")
q_lo.set_power(15)

In [176]:
vna.set_nop(1)
vna.set_center(7.04720e9)
vna.sweep_continuous()

In [178]:
q_awg.output_pulse_sequence(q_pb.add_zero_pulse(20).build())
ro_awg.output_pulse_sequence(ro_pb.add_zero_pulse(20).build())

In [182]:
q_awg.output_pulse_sequence(q_pb.add_sine_pulse(20,0).build())
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(20,5).build())

### Rabi 

In [10]:
# reload(lib2.Measurement)
reload(lib2.TimeResolvedMeasurement)
from lib2.TimeResolvedMeasurement import DispersiveRabiOscillations

In [8]:
# Measurement._actual_devices.keys()

In [11]:
DRO = DispersiveRabiOscillations("VI-ac-stark", "Xmon Al BMSTU S444 2",
            vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg', current_src_name='yok3')
vna_parameters = {"bandwidth":1, "freq_limits":(7040e6, 7052.5e6), "nop":1, "averages":1}
q_lo_parameters = {}
current = 1.04e-3
q_frequency = 7.9e9
excitation_durations = linspace(1, 1000, 100)
rabi_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, "repetition_period":8000}
DRO.setup_control_parameters(vna_parameters, q_lo_parameters, ro_cal, q_cal, 
                             q_frequency, current, rabi_sequence_parameters, excitation_durations)

Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of 1.04 mA
Detected frequency is 7.04704 GHz, at 0.04 mU and 24.85 degrees


In [12]:
DRO.launch()

Started at:  2017-04-26 09:14:29.125922


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 2 m 14.69 s, excitation duration: 3.441e+02 ns       

In [22]:
result.visualize()

In [48]:
dso.set_time_range(10e-6)
dso.set_time_offset(1e-6)
dso.digitize()
times, channel_data = dso.get_data(Channel.ONE, Channel.TWO)
axes = flatten(plt.subplots(2,1, sharex=True, figsize=(13,5))[1])
colors = ["C0", "C1", "C2", "C3"]
for idx, ax in enumerate(axes):
    ax.plot(times, channel_data[idx], color=colors[idx])
    ax.minorticks_on()
    ax.grid(which="both")
    ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.tight_layout()

NameError: name 'dso' is not defined

In [355]:
q_frequency = 7.9e9
excitation_times = linspace(1, 1500, 50)
excitation_freqs = linspace(q_frequency-15e6, q_frequency+15e6, 50)#[q_frequency]#

start_time = dt.now()
print("Started at", start_time)

rabi_shevrons = []
phases = []
for i, excitation_freq in enumerate(excitation_freqs):
    q_lo.set_frequency(excitation_freq+if_freq)
    z_data = []
    for j, t in enumerate(excitation_times):
        
        generate_pulses_rabi(t)
#         sleep(0.5)
        vna.avg_clear()
        vna.prepare_for_stb()
        vna.sweep_single()
        vna.wait_for_stb()
        s_data = vna.get_sdata()
        z_data.append(mean(s_data))
        phases.append(median(angle(s_data)))
        
        part_complete = (i*len(excitation_times)+j+1)/len(excitation_times)/len(excitation_freqs)
        elapsed_time = (dt.now() - start_time).total_seconds()
        time_left = format_time_delta(elapsed_time/part_complete-elapsed_time)
#         cla()
#         plot(excitation_times[:j+1], absolute(z_data))
#         plt.pause(0.01)
        print("\r%.5f%% Time: %d ns, freq: %.5f GHz. Time left: %s    "%(part_complete*100, t,
                excitation_freq/1e9, time_left), end="")
    rabi_shevrons.append(z_data)
    pcolormesh(absolute(rabi_shevrons).T)
    plt.pause(0.01)

rabi_shevrons = array(rabi_shevrons)

Started at 2017-04-25 10:47:28.472245
2.00000% Time: 1500 ns, freq: 7.88500 GHz. Time left: 1 h 30 m 43.77 s    

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


100.00000% Time: 1500 ns, freq: 7.91500 GHz. Time left: 0 h 0 m 0.0 s      

In [372]:
pcolormesh(excitation_freqs, excitation_times, absolute(rabi_shevrons).T)

In [339]:
def theoretical_rabi(t, A, gamma_R, Omega_R, offset):
    return A*exp(-gamma_R*t)*cos(Omega_R*t)+offset

In [340]:
opt_params = curve_fit(theoretical_rabi, excitation_times[:], abs(rabi_shevrons)[0][:], (.1, 1.5e-3, 5e-2, -0.96))[0]
opt_params

array([ -4.66659262e-05,   5.67426473e-04,   4.72207804e-02,
         1.43297208e-04])

In [371]:
plot(excitation_times, abs(rabi_shevrons)[0])
plot(excitation_times, theoretical_rabi(excitation_times, *opt_params))
xlabel("Excitation time [ns]")
ylabel("VNA amplitude [a.u.]")
bbox_props = dict(boxstyle="round", fc="white", ec="black", lw=1)
annotate("$T_R=%.2f\ \mu$s\n$\Omega_R/2\pi = %.2f$ MHz"%(1/opt_params[1]/1e3, opt_params[2]*1e3/2/pi),
             (mean(gca().get_xlim()),  abs(rabi_shevrons)[0][0]), bbox=bbox_props, ha="center")

In [360]:
from scipy.fftpack import fft, fftshift, fftfreq

In [384]:
start = 33
freqs = fftshift(fftfreq(len(excitation_times), excitation_times[1]-excitation_times[0]))[start:]
pcolormesh(freqs*1e3, excitation_freqs, fftshift(abs(fft(abs(rabi_shevrons))), axes=1)[:,start:])


In [298]:
import pickle

In [357]:
with open("data/Xmon Al BMSTU S444 2/rabi-higher-shevrons.pkl", "wb") as f:
    pickle.dump((excitation_freqs, excitation_times, rabi_shevrons), f)

### Decay

In [304]:
t_pi = 206

In [306]:
delay = 5000
def generate_pulses(delay):
    q_pb.add_zero_pulse(400).add_sine_pulse(t_pi, 5).add_zero_pulse(1000).add_zero_pulse(10000-1000-t_pi-400)
    q_awg.output_pulse_sequence(q_pb.build())

    ro_pb.add_zero_pulse(t_pi+delay).add_dc_pulse(1000,5).add_zero_pulse(1)
    ro_awg.output_pulse_sequence(ro_pb.build())
generate_pulses(delay)

In [308]:
dso.set_time_range(10e-6)
dso.set_time_offset(1e-6)
dso.digitize()
times, channel_data = dso.get_data(Channel.ONE, Channel.TWO)
axes = flatten(plt.subplots(2,1, sharex=True, figsize=(13,5))[1])
colors = ["C0", "C1", "C2", "C3"]
for idx, ax in enumerate(axes):
    ax.plot(times, channel_data[idx], color=colors[idx])
    ax.minorticks_on()
    ax.grid(which="both")
    ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.tight_layout()

In [310]:
delays = linspace(0, 7500, 75)

start_time = dt.now()
print("Started at", start_time)

z_data = []
for j, delay in enumerate(delays):
    
    generate_pulses(delay)
    
    vna.avg_clear()
    vna.prepare_for_stb()
    vna.sweep_single()
    vna.wait_for_stb()
    z_data.append(median(abs(vna.get_sdata())))

    part_complete = (j+1)/len(delays)
    elapsed_time = (dt.now() - start_time).total_seconds()
    time_left = format_time_delta(elapsed_time/part_complete-elapsed_time)
    plt.cla()
    plot(delays[:j+1],z_data)
    plt.pause(0.01)
    print("\r%.5f%% Time: %d ns, freq: %.5f GHz. Time left: %s    "%(part_complete*100, delay,
            excitation_freq/1e9, time_left), end="")

z_data = array(z_data)

Started at 2017-04-24 19:42:56.095042


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


100.00000% Time: 7500 ns, freq: 7.89874 GHz. Time left: 0 h 0 m 0.0 s     

In [311]:
def theoretical_decay(t, gamma, A, offset):
    return A*exp(-gamma*t) + offset

In [322]:
opt_params = curve_fit(theoretical_decay, delays[:], z_data[:], p0=(0.001, 1, -0.8))[0]

In [323]:
plot(delays, z_data)
plot(delays, theoretical_decay(delays, *opt_params))
xlabel("Readout delay [ns]")
ylabel("VNA amplitude [a.u.]")
bbox_props = dict(boxstyle="round", fc="white", ec="black", lw=1)
annotate("$T_1=%.2f\ \mu$s"%(1/opt_params[0]/1e3), (mean(gca().get_xlim()), z_data[0]*1.01), bbox=bbox_props, ha="center")

In [314]:
with open("./data/Xmon Al BMSTU S444 2/decay-VI", "w+b") as f:
    pickle.dump((delays, z_data), f)

In [324]:
q_lo.visa_instr.close()

## Loading

In [ ]:
result = MeasurementResult.load("Xmon Al BMSTU S444 2", "VI-powerscan")

In [ ]:
result.set_phase_units("deg")

In [ ]:
result.visualize()

## Measurement class testing

In [392]:
from importlib import reload

In [405]:
reload(lib2.Measurement)
from lib2.Measurement import *

In [408]:
meas = Measurement("TEST", "TEST", ['vna2'])

The device vna2 is detected as PNA-L2


In [409]:
meas._actual_devices["vna2"]

In [75]:
meas._mxg.visa_instr.close()

In [12]:
meas._vna2.get_power()

0.0

In [410]:
q_lo.visa_instr.close()

In [59]:
Agilent_EXA

NameError: name 'Agilent_EXA' is not defined

In [ ]:
from drivers import Agilent_PNA_L

In [57]:
mxg

TypeError: __repr__ returned non-string (type TCPIPInstrument)

In [ ]:
import drivers

In [ ]:
drivers.

In [ ]:
from drivers import *

In [ ]:
ю

In [ ]:
./

In [ ]:
from drivers import *

In [391]:
reload(lib2.Measurement)

AttributeError: module 'lib2' has no attribute 'Measurement'

In [ ]:
Agilent_EXA

In [386]:
import pyvisa

In [387]:
rm = pyvisa.ResourceManager()
temp_list = list(rm.list_resources_info().values())

devs_info = \
                [(list(temp_list)[i])[3:5] for i in range(len(temp_list))]

In [389]:
devs_info[1]

('TCPIP0::10.55.70.46::hislip0::INSTR', 'PNA-L1')